# Data Compression Howework #2 
Adaptive Huffman Coding

In [1]:
import sys

In [2]:
class Node(object):
    
    def __init__(self, parent=None, left=None, right=None, weight=0, symbol=''):
        super(Node, self).__init__()
        self._parent = parent
        self._left = left
        self._right = right
        self._weight = weight
        self._symbol = symbol

    @property
    def parent(self):
        return self._parent

    @parent.setter
    def parent(self, parent):
        self._parent = parent

    @property
    def left(self):
        return self._left

    @left.setter
    def left(self, left):
        self._left = left

    @property
    def right(self):
        return self._right

    @right.setter
    def right(self, right):
        self._right = right

    @property
    def weight(self):
        return self._weight

    @weight.setter
    def weight(self, weight):
        self._weight = weight

    @property
    def symbol(self):
        return self._symbol

    @symbol.setter
    def symbol(self, symbol):
        self._symbol = symbol

In [3]:
class FGK(object):
    
    # 생성자
    def __init__(self):
        super(FGK, self).__init__()
        
        # 초기 NYT 노드 설정
        self.NYT = Node(symbol="NYT")
        self.root = self.NYT
        self.nodes = []
        self.seen = [None] * 256
        
    # 코드 구하기
    def get_code(self, s, node, code=''):
        
        # 터미널 노드이면 
        if node.left is None and node.right is None:
            return code if node.symbol == s else ''
        
        # 터미널 노드가 아니면
        else:
            temp = ''
            
            # 왼쪽 노드가 있으면
            if node.left is not None:
                temp = self.get_code(s, node.left, code+'0')
                
            # 오른쪽 노드가 있으면
            if not temp and node.right is not None:
                temp = self.get_code(s, node.right, code+'1')
                
            return temp

    
    # 가장 큰 값을 가진 노드 찾기
    def find_largest_node(self, weight):
        
        for n in reversed(self.nodes):
            if n.weight == weight:
                return n

            
    # swapping
    def swap_node(self, n1, n2):
        i1, i2 = self.nodes.index(n1), self.nodes.index(n2)
        self.nodes[i1], self.nodes[i2] = self.nodes[i2], self.nodes[i1]
        
        tmp_parent = n1.parent
        n1.parent = n2.parent
        n2.parent = tmp_parent

        if n1.parent.left is n2:
            n1.parent.left = n1
        else:
            n1.parent.right = n1

        if n2.parent.left is n1:
            n2.parent.left = n2
        else:
            n2.parent.right = n2

            
    # 새로운 노드 삽입
    def insert(self, s):
        node = self.seen[ord(s)]

        # 노드의 문자가 원래 있는 문자가 아니라면 
        if node is None:
            
            # 새로운 노드 생성
            spawn = Node(symbol=s, weight=1)
            internal = Node(symbol='', weight=1, parent=self.NYT.parent,
                left=self.NYT, right=spawn)
            spawn.parent = internal
            self.NYT.parent = internal

            # internal의 부모 노드가 있다면
            if internal.parent is not None:
                internal.parent.left = internal
            
            # 없다면 루트로 
            else:
                self.root = internal

            # 노드 추가
            self.nodes.insert(0, internal)
            self.nodes.insert(0, spawn)

            self.seen[ord(s)] = spawn
            node = internal.parent

        while node is not None:
            largest = self.find_largest_node(node.weight)

            if (node is not largest and node is not largest.parent and
                largest is not node.parent):
                self.swap_node(node, largest)

            node.weight = node.weight + 1
            node = node.parent

            
    # encoding
    def encode(self, text):
        result = ''

        for s in text:
            # 문자를 아스키 코드로 바꾸는 함수 ord
            if self.seen[ord(s)]:
                
                # root로 부터 code 구하기
                result += self.get_code(s, self.root)
                
            else:
                
                # NYT의 code 구하기
                result += self.get_code('NYT', self.root)
                result += bin(ord(s))[2:].zfill(8) # zero padding

            self.insert(s)

        return result

    
    # ascii code -> Symbol
    def get_symbol_by_ascii(self, bin_str):
        return chr(int(bin_str, 2))

    
    # decoding
    def decode(self, text):
        result = ''

        symbol = self.get_symbol_by_ascii(text[:8])
        result += symbol
        self.insert(symbol)
        node = self.root

        i = 8
        while i < len(text):
            node = node.left if text[i] == '0' else node.right
            symbol = node.symbol

            if symbol:
                if symbol == 'NYT':
                    symbol = self.get_symbol_by_ascii(text[i+1:i+9])
                    i += 8

                result += symbol
                self.insert(symbol)
                node = self.root

            i += 1

        return result


In [4]:
# encoding
with open('text.txt') as f:
    text = f.read()
    print(text)
fgk = FGK()
result = fgk.encode(text)
print(result)
with open('encoded_text.txt', 'w') as f:
    f.write(result)

In this chapter we describe a very popular coding algorithm called the Huffman coding algorithm. We first present a procedure for building Huffman codes when the probability model for the source is known, then a procedure for building codes when the source statistics are unknown. We also describe a few techniques for code design that are in some sense similar to the Huffman coding approach. Finally, we give some examples of using the Huffman code for image compression, audio compression, and text compression
010010010011011100000100000100011101000000110100011000110100110000111001110001000110001110111100011000011010001110000010100000110010101000011100101110010001110111001111100000011001001101010000110001001111000001100010011111110000001010001110110011100111111000111100111101010001000011011110001000000011101011001100011011000100101111001001111110000000110000011110000110011111001101010011010001101111100011110111101111010000011011011100110100110001000011111001011000110010111111011011100010

In [5]:
# print frequency and code(final tree) 
for i in range(len(fgk.nodes)):
    if str(fgk.nodes[i].symbol) != '':
        print(str(fgk.nodes[i].symbol) + " : " + str(fgk.nodes[i].weight).rjust(2) + " , " + str(fgk.get_code(fgk.nodes[i].symbol, fgk.root)).rjust(10))

q :  1 , 1000111001
F :  1 ,  100011101
I :  1 ,  111100100
x :  2 ,  111100101
v :  2 ,   10001111
W :  2 ,   11110000
k :  2 ,   11110001
y :  3 ,   11110011
. :  3 ,    0010110
, :  4 ,    0010111
H :  4 ,    1000110
b :  6 ,     001010
w :  7 ,     100010
g : 11 ,     111101
p : 13 ,      00100
l : 14 ,      01110
m : 15 ,      01111
u : 15 ,      10000
f : 16 ,      10100
h : 16 ,      10101
d : 18 ,      11100
c : 21 ,      11101
t : 23 ,      11111
a : 27 ,       0011
r : 28 ,       0100
n : 29 ,       0101
s : 29 ,       0110
i : 31 ,       1001
o : 36 ,       1011
e : 51 ,        000
  : 82 ,        110


In [6]:
# decoding
with open('encoded_text.txt', 'r') as f:
    text = f.read()
    print(text)
result2 = FGK().decode(text)
print("result of decoding :",result2)

0100100100110111000001000001000111010000001101000110001101001100001110011100010001100011101111000110000110100011100000101000001100101010000111001011100100011101110011111000000110010011010100001100010011110000011000100111111100000010100011101100111001111110001111001111010100010000110111100010000000111010110011000110110001001011110010011111100000001100000111100001100111110011010100110100011011111000111101111011110100000110110111001101001100010000111110010110001100101111110110111000100100010001100100000110011011111001001001101110010110011011100111000111101000110011010100101011011111110010111111111111100000000100000101110110000111000101011110111100000110000111000000011110011010111101100001110111101010111001111101010010010100000111111010010010010110001100110001001011110100101100100011010111101010110101110110011010100010101101110011111010010110111011101110101000000111101110110010111010100111100101100010101101111110100011010010001000101100100110110101110110001001000111101111110100111101111101